# 02_3_itb_link_mapping

#### Crawling for knowing this architecture a website

this code for knowing an architecture a website yaitu https://www.itb.ac.id/ yang mana kita membaca setiap href yang ada di setiap page kemudian dipilih hanya page yang berdomain internal yaitu itb.ac.id kemudian didapatkan link keluarnya nah dari link keluarnya itu kemudian di search link apa aja yang ada dalam page tersebut, dilakukan berulang kali untuk semua link hrefnya yang ada dalam page

Hasil dari this code dapat dilihat dalam link [architecture web](https://drive.google.com/file/d/1TISHRRxZ47QrKTTlm77P-rNRauj611IV/view?usp=drive_link)

In [ ]:
! pip install requests
! pip install bs4
! pip install pandas

In [16]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import pandas as pd
import time
from collections import deque

In [17]:


def crawl_itb_website(max_pages=100):
    """
    Fungsi untuk melakukan crawling pada website ITB dan memetakan semua link internal
    """
    # Header untuk mensimulasikan browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    # URL awal (homepage ITB)
    start_url = "https://www.itb.ac.id/"
    
    # Menyimpan base domain untuk memfilter link internal
    base_domain = urlparse(start_url).netloc
    
    # Inisialisasi struktur data
    visited = set()  # Halaman yang sudah dikunjungi
    to_visit = deque([start_url])  # Antrian halaman yang akan dikunjungi
    link_data = []  # Data link yang akan disimpan
    
    # Counter untuk ID
    link_id = 1
    
    # Loop sampai tidak ada lagi halaman untuk dikunjungi atau mencapai batas maksimal
    while to_visit and len(visited) < max_pages:
        # Ambil URL berikutnya dari antrian
        current_url = to_visit.popleft()
        
        # Skip jika sudah dikunjungi
        if current_url in visited:
            continue
            
        print(f"Mengunjungi: {current_url}")
        
        try:
            # Request halaman
            response = requests.get(current_url, headers=headers, timeout=10)
            response.raise_for_status()
            
            # Parse HTML
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Temukan semua link di halaman
            links = soup.find_all('a', href=True)
            
            # Filter dan proses setiap link
            for link in links:
                href = link['href']
                
                # Skip anchor links dan javascript
                if href.startswith('#') or href.startswith('javascript:'):
                    continue
                
                # Buat URL absolut
                absolute_url = urljoin(current_url, href)
                
                # Parse URL untuk mendapatkan komponen
                parsed_url = urlparse(absolute_url)
                
                # Hanya pertimbangkan link internal (dalam domain ITB)
                if parsed_url.netloc == base_domain:
                    # Tambahkan ke data
                    link_data.append({
                        'id': link_id,
                        'Halaman yang Dikunjungi (Page)': current_url,
                        'Link yang Ditemukan (Link Keluar)': absolute_url
                    })
                    link_id += 1
                    
                    # Tambahkan ke antrian jika belum dikunjungi
                    if absolute_url not in visited and absolute_url not in to_visit:
                        to_visit.append(absolute_url)
            
            # Tandai halaman saat ini sebagai telah dikunjungi
            visited.add(current_url)
            
            # Delay untuk tidak membebani server
            time.sleep(1)
            
        except requests.exceptions.RequestException as e:
            print(f"Error saat mengakses {current_url}: {e}")
            continue
        except Exception as e:
            print(f"Error tidak terduga: {e}")
            continue
    
    # Buat DataFrame dari data yang dikumpulkan
    df = pd.DataFrame(link_data)
    return df

# Menjalankan crawler
if __name__ == "__main__":
    print("Memulai crawling website ITB...")
    hasil_crawling = crawl_itb_website(max_pages=100)
    
    if not hasil_crawling.empty:
        # Simpan hasil ke CSV
        hasil_crawling.to_csv('itb_link_mapping.csv', index=False)
        print(f"Berhasil memetakan {len(hasil_crawling)} link. Data disimpan ke itb_link_mapping.csv")
        
        # Tampilkan preview
        print("\nPreview data:")
        print(hasil_crawling.head(10))
    else:
        print("Tidak ada data yang berhasil dikumpulkan.")

Memulai crawling website ITB...
Mengunjungi: https://www.itb.ac.id/
Mengunjungi: https://www.itb.ac.id/myitb
Mengunjungi: https://www.itb.ac.id/staff/
Mengunjungi: https://www.itb.ac.id/kehidupan-kampus
Mengunjungi: https://www.itb.ac.id/alumni
Mengunjungi: https://www.itb.ac.id/berita/qs-wur-2025-itb-raih-peringkat-256-dunia/60769
Mengunjungi: https://www.itb.ac.id/contact
Error saat mengakses https://www.itb.ac.id/contact: HTTPSConnectionPool(host='www.itb.ac.id', port=443): Read timed out. (read timeout=10)
Mengunjungi: https://www.itb.ac.id/directories
Mengunjungi: https://www.itb.ac.id/agenda/academic
Mengunjungi: https://www.itb.ac.id/staff/campus-life
Error saat mengakses https://www.itb.ac.id/staff/campus-life: 404 Client Error: Not Found for url: https://itb.ac.id:443/staff/campus-life
Mengunjungi: https://www.itb.ac.id/staf/listby/
Mengunjungi: https://www.itb.ac.id/researchers-profilei
Error saat mengakses https://www.itb.ac.id/researchers-profilei: 404 Client Error: Not Fou